In [1]:
import pandas as pd
from os import walk
from datetime import datetime

In [2]:
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)
  
    if (a_set & b_set):
        return(list(a_set & b_set))
    else:
        print("No common elements")

In [3]:
enr_up_files = []
new_enr_files = []
for _ in walk("."):
    if (_[0] == '.'):
        for file in _[2]:
            if (file[:11] == 'enr_update_'):
                enr_up_files.append(datetime.strptime(file.strip('enr_update_').rstrip('.csv'), '%m_%d_%Y'))
            if (file[:8] == 'new_enr_'):
                new_enr_files.append(datetime.strptime(file.strip('new_enr_').rstrip('.csv'), '%m_%d_%Y'))

enr_update_file = f"enr_update_{max(enr_up_files).strftime('%m_%d_%Y')}.csv"
enr_update = pd.read_csv(enr_update_file)

new_enr_file = f"new_enr_{max(new_enr_files).strftime('%m_%d_%Y')}.csv"
new_enr = pd.read_csv(new_enr_file)

match_table = pd.read_csv('debugging_output/__match_table.csv')
import_nsc = pd.read_csv('data/import_nsc_output.csv')
import_nsc.dropna(axis = 0, subset = ['Enrollment Status'], inplace = True)

In [4]:
columns = list(match_table.columns.copy()) + ['Enrollment Status']
match_table = match_table.join(import_nsc[common_member(match_table.columns, import_nsc.columns) + ['Enrollment Status']], rsuffix = '_import')[columns]

In [5]:
enr_update_columns = list(enr_update.columns.copy()) + ['Enrollment_Status__c']
enr_update_status = enr_update.merge(match_table[['Id','Enrollment Status']], on = ['Id'], how = 'left')

In [7]:
enr_update_status.columns = enr_update_columns

In [8]:
enr_update_status.to_csv(f"data/{'_enrollment_status.'.join(enr_update_file.split('.'))}", index = False)

In [9]:
new_enr_columns = list(new_enr.columns) + ['Enrollment_Status__c']
new_enr_status = new_enr.merge(match_table[['Student__c','College__c','End_Date__c','Date_Last_Verified__c','Enrollment Status']] ,how='left')

In [10]:
new_enr_status.columns = new_enr_columns

In [11]:
new_enr_status.to_csv(f"data/{'_enrollment_status.'.join(new_enr_file.split('.'))}", index = False)